In [1]:
!git clone https://github.com/syrgkanislab/npiv_functionals.git

Cloning into 'npiv_functionals'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 195 (delta 99), reused 138 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (195/195), 669.79 KiB | 22.33 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [2]:
%cd npiv_functionals

/content/npiv_functionals


In [3]:
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [4]:
import os
import sys
sys.path.append(os.path.abspath('.'))

In [5]:
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import mean_ci
from mliv.neuralnet import AGMMEarlyStop as AGMM
from mliv.neuralnet.moments import avg_small_diff
from sklearn.ensemble import RandomForestRegressor
import joblib
from joblib import Parallel, delayed
from mliv.cct.mc2 import MC2
from mliv.rkhs import ApproxRKHSIVCV
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import FeatureUnion, Pipeline
import scipy

In [6]:
# average finite difference moment
def moment_fn(x, test_fn):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return (test_fn(t1) - test_fn(t0)) / (2 * epsilon)

In [7]:
def moment_evals(x):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return t0, t1

In [8]:
it = 0
n = 5000
mc2_gen = MC2(n, 100, None, dimension=10, corr=0.5)
npvec, *_ = mc2_gen.data(it)
Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
n_z = Z.shape[1]
n_x = X.shape[1]

In [9]:
Z_train, Z_val, X_train, X_val, Y_train, Y_val = train_test_split(
        Z, X, Y, test_size=.5, shuffle=True)

ztrans = Nystroem(n_components=100)
xtrans = Nystroem(n_components=100)
# ztrans = PolynomialFeatures(degree=2)
# xtrans = PolynomialFeatures(degree=2)
# ztrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
# xtrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])
Psi = ztrans.fit_transform(Z_train)
xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
Phi = xtrans.transform(X_train)
mPhi = moment_fn(X_train, xtrans.transform)

In [10]:
CovPsi = Psi.T @ Psi
CovPhiPsi = Phi.T @ Psi
Phival = xtrans.transform(X_val)
Psival = ztrans.transform(Z_val)
mPhival = moment_fn(X_val, xtrans.transform)
moment_val = np.mean(mPhival, axis=0)

best_violation = np.inf
for alpha in np.logspace(-6, 1, 5):
    regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
    Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
    for beta in np.logspace(-6, 1, 5):
        xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        for gamma in np.logspace(-6, 1, 5):
            qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

            representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
            violation = np.linalg.norm(moment_val - representer_val, ord=2)
            if violation <= best_violation:
                best_alpha = alpha
                best_beta = beta
                best_gamma = gamma
                best_violation = violation

In [11]:
alpha = best_alpha
beta = best_beta
gamma = best_gamma
regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

In [12]:
best_alpha, best_beta, best_gamma, best_violation

(1e-06, 0.0031622776601683794, 0.0031622776601683794, 1.0404707570575489)

In [13]:
agmm = ApproxRKHSIVCV(n_components=200)
agmm.fit(Z_train, X_train, Y_train)

In [14]:
direct = moment_fn(X_val, agmm.predict).flatten()
residual = (Y_val - agmm.predict(X_val)).flatten()
qvalues = Psival @ qparam
pseudo = direct + qvalues * residual

reg = mean_ci(direct)
dr = mean_ci(pseudo)
ipw = mean_ci(qvalues * Y_val.flatten())
reg, ipw, dr

((1.2141849059398333, 1.0974348660172508, 1.3309349458624158),
 (0.4970061061199458, -0.3013469088314763, 1.295359121071368),
 (1.1251425038796286, 0.850870651162335, 1.3994143565969221))

In [15]:
xivalues = xtrans.transform(X_val) @ xi
coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
pseudo_tmle = direct + coef * (mPhival @ xi)
pseudo_tmle += qvalues * (residual - coef * xivalues)
tmle = mean_ci(pseudo_tmle)
tmle

(1.116385622031688, 0.8455549641215679, 1.387216279941808)

In [16]:
def exp_sample_split(it, n, dim, corr):
    mc2_gen = MC2(n, 100, None, dimension=dim, corr=corr)
    npvec, *_ = mc2_gen.data(it)
    Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
    n_z = Z.shape[1]
    n_x = X.shape[1]

    Z_train, Z_val, X_train, X_val, Y_train, Y_val = train_test_split(
        Z, X, Y, test_size=.5, shuffle=True)

    ztrans = PolynomialFeatures(degree=3)
    xtrans = PolynomialFeatures(degree=3)
    #     ztrans = Nystroem(n_components=100)
    #     xtrans = Nystroem(n_components=100)
    #     ztrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=5))])
    #     xtrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=5))])
    ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
    xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])

    Psi = ztrans.fit_transform(Z_train)
    Phi = xtrans.fit_transform(X_train)
    mPhi = moment_fn(X_train, xtrans.transform)

    CovPsi = Psi.T @ Psi
    CovPhiPsi = Phi.T @ Psi
    Phival = xtrans.transform(X_val)
    Psival = ztrans.transform(Z_val)
    mPhival = moment_fn(X_val, xtrans.transform)
    moment_val = np.mean(mPhival, axis=0)

    best_violation = np.inf
    for alpha in np.logspace(-6, 1, 5):
        regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
        Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
        for beta in np.logspace(-6, 1, 5):
            xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
            for gamma in np.logspace(-6, 1, 5):
                qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

                # calculating the violation in the riesz representation property for each feature
                #  E[m(W; phi)] = E[q(Z) * phi(X)]
                # for every feature phi.
                representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
                violation = np.linalg.norm(moment_val - representer_val, ord=2)
                if violation <= best_violation:
                    best_alpha = alpha
                    best_beta = beta
                    best_gamma = gamma
                    best_violation = violation

    alpha = best_alpha
    beta = best_beta
    gamma = best_gamma
    regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
    Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
    xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
    qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

    agmm = ApproxRKHSIVCV(n_components=200)
    agmm.fit(Z_train, X_train, Y_train)

    direct = moment_fn(X_val, agmm.predict).flatten()
    residual = (Y_val - agmm.predict(X_val)).flatten()
    qvalues = Psival @ qparam
    pseudo = direct + qvalues * residual

    reg = mean_ci(direct)
    dr = mean_ci(pseudo)
    ipw = mean_ci(qvalues * Y_val.flatten())

    xivalues = Phival @ xi
    coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
    pseudo_tmle = direct + coef * (mPhival @ xi)
    pseudo_tmle += qvalues * (residual - coef * xivalues)
    tmle = mean_ci(pseudo_tmle)

    return dr, tmle, ipw, reg

In [17]:
from sklearn.model_selection import KFold

def exp(it, n, dim, corr, fname='cct', iv_strength=None):
    np.random.seed(it)
    if fname == 'cct':
        mc2_gen = MC2(n, 100, None, dimension=dim, corr=corr)
        npvec, *_ = mc2_gen.data(it)
        Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
    else:
        Z, X, Y, _ = get_data(n, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5)

    direct = np.zeros(n)
    residual = np.zeros(n)
    qvalues = np.zeros(n)
    xivalues = np.zeros(n)
    mxivalues = np.zeros(n)

    for train, test in KFold(n_splits=5, shuffle=True).split(Z):
        Z_train, Z_val, X_train, X_val, Y_train, Y_val = Z[train], Z[test], X[train], X[test], Y[train], Y[test]

        ztrans = Nystroem(n_components=100)
        xtrans = Nystroem(n_components=100)
        ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
        xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])

        Psi = ztrans.fit_transform(Z_train)
        xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
        Phi = xtrans.transform(X_train)
        mPhi = moment_fn(X_train, xtrans.transform)

        CovPsi = Psi.T @ Psi
        CovPhiPsi = Phi.T @ Psi
        Phival = xtrans.transform(X_val)
        Psival = ztrans.transform(Z_val)
        mPhival = moment_fn(X_val, xtrans.transform)
        moment_val = np.mean(mPhival, axis=0)

        best_violation = np.inf
        for alpha in np.logspace(-6, 1, 5):
            regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
            Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
            for beta in np.logspace(-6, 1, 5):
                xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
                for gamma in np.logspace(-6, 1, 5):
                    qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

                    # calculating the violation in the riesz representation property for each feature
                    #  E[m(W; phi)] = E[q(Z) * phi(X)]
                    # for every feature phi.
                    representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
                    violation = np.linalg.norm(moment_val - representer_val, ord=2)
                    if violation <= best_violation:
                        best_alpha = alpha
                        best_beta = beta
                        best_gamma = gamma
                        best_violation = violation

        alpha = best_alpha
        beta = best_beta
        gamma = best_gamma
        regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
        Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
        xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

        agmm = ApproxRKHSIVCV(n_components=200)
        agmm.fit(Z_train, X_train, Y_train)

        direct[test] = moment_fn(X_val, agmm.predict).flatten()
        residual[test] = (Y_val - agmm.predict(X_val)).flatten()
        qvalues[test] = Psival @ qparam
        xivalues[test] = Phival @ xi
        mxivalues[test] = mPhival @ xi

    pseudo = direct + qvalues * residual

    reg = mean_ci(direct)
    dr = mean_ci(pseudo)
    ipw = mean_ci(qvalues * Y.flatten())

    coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
    pseudo_tmle = direct + coef * mxivalues
    pseudo_tmle += qvalues * (residual - coef * xivalues)
    tmle = mean_ci(pseudo_tmle)

    return dr, tmle, ipw, reg

In [34]:
def get_result_dict(results, true, alpha=0.99):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        if method == 'ipw':
            continue
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        confidence = alpha
        data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov': cov,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}',
                'std': f'{np.std(data[:, 0]):.3f}'}
    return df

In [19]:
import pandas as pd

true = 1.0

for n in [1000, 5000]:
    for n_x in [0, 5, 10]:
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            print(n, n_x, corr)
            results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, n_x, corr)
                                                            for it in range(100))
            joblib.dump(results, f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            display(df)

1000 0 0.0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.2min finished


,dr,tmle,ipw,direct
cov,98,92,NA,NA
rmse,0.072,0.255,0.154,0.051
bias,0.000,0.025,0.103,0.018
std,0.072,0.254,0.114,0.047


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


1000 5 0.0


...

,dr,tmle,ipw,direct
cov,89,86,NA,NA
rmse,0.140,4.894,0.117,0.106
bias,0.089,0.477,0.092,0.082
std,0.108,4.871,0.072,0.067


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


5000 10 0.5


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished


,dr,tmle,ipw,direct
cov,94,91,NA,NA
rmse,0.141,0.485,0.097,0.144
bias,0.071,0.118,0.037,0.114
std,0.122,0.470,0.089,0.088


In [38]:
true = 1.0

res = {}
for n in [1000, 5000]:
    res[f'$n={n}$'] = {}
    for n_x in [0, 5, 10]:
        res[f'$n={n}$'][f'$n_x={n_x}$'] = {}
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            results = joblib.load(f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            res[f'$n={n}$'][f'$n_x={n_x}$'][f'$\\rho={corr}$'] = pd.DataFrame(get_result_dict(results, true))
        res[f'$n={n}$'][f'$n_x={n_x}$'] = pd.concat(res[f'$n={n}$'][f'$n_x={n_x}$'], sort=False)
    res[f'$n={n}$'] = pd.concat(res[f'$n={n}$'], sort=False)
res = pd.concat(res, sort=False).unstack(level=3)

In [39]:
res

dr                      tmle                 \
                              cov   rmse   bias    std  cov    rmse   bias   
$n=1000$ $n_x=0$  $\rho=0.0$   98  0.072  0.000  0.072   92   0.255  0.025   
         $n_x=5$  $\rho=0.0$   97  0.148  0.010  0.148   94   2.482  0.204   
                  $\rho=0.5$   96  0.166  0.045  0.160   94   0.741  0.100   
         $n_x=10$ $\rho=0.0$  100  0.169  0.025  0.167   97   0.549  0.023   
                  $\rho=0.5$   98  0.269  0.178  0.202   88   1.233  0.169   
$n=5000$ $n_x=0$  $\rho=0.0$   99  0.032  0.010  0.030   97   0.085  0.023   
         $n_x=5$  $\rho=0.0$   97  0.080  0.038  0.070   92   0.339  0.040   
                  $\rho=0.5$   96  0.086  0.005  0.085   94  11.871  1.189   
         $n_x=10$ $\rho=0.0$   89  0.140  0.089  0.108   86   4.894  0.477   
                  $\rho=0.5$   94  0.141  0.071  0.122   91   0.485  0.118   

                                     direct                       
                                 std    cov   rmse   bias    std  
$n=1000$ $n_x=0$  $\rho=0.0$   0.254     NA  0.051  0.018  0.047  
         $n_x=5$  $\rho=0.0$   2.473     NA  0.103  0.035  0.097  
                  $\rho=0.5$   0.734     NA  0.104  0.047  0.093  
         $n_x=10$ $\rho=0.0$   0.548     NA  0.271  0.259  0.080  
                  $\rho=0.5$   1.222     NA  0.106  0.007  0.106  
$n=5000$ $n_x=0$  $\rho=0.0$   0.082     NA  0.026  0.006  0.025  
         $n_x=5$  $\rho=0.0$   0.336     NA  0.068  0.037  0.057  
                  $\rho=0.5$  11.811     NA  0.069  0.021  0.065  
         $n_x=10$ $\rho=0.0$   4.871     NA  0.106  0.082  0.067  
                  $\rho=0.5$   0.470     NA  0.144  0.114  0.088

In [40]:
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='lll||llll|llll|llll|',
                   multicolumn_format='c|'))

\begin{tabular}{lll||llll|llll|llll|}
\toprule
 &  &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{direct} \\
 &  &  & cov & rmse & bias & std & cov & rmse & bias & std & cov & rmse & bias & std \\
\midrule
\multirow[t]{5}{*}{\textbf{$n=1000$}} & \textbf{$n_x=0$} & \textbf{$\rho=0.0$} & 98 & 0.072 & 0.000 & 0.072 & 92 & 0.255 & 0.025 & 0.254 & NA & 0.051 & 0.018 & 0.047 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n_x=5$}} & \textbf{$\rho=0.0$} & 97 & 0.148 & 0.010 & 0.148 & 94 & 2.482 & 0.204 & 2.473 & NA & 0.103 & 0.035 & 0.097 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 96 & 0.166 & 0.045 & 0.160 & 94 & 0.741 & 0.100 & 0.734 & NA & 0.104 & 0.047 & 0.093 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n_x=10$}} & \textbf{$\rho=0.0$} & 100 & 0.169 & 0.025 & 0.167 & 97 & 0.549 & 0.023 & 0.548 & NA & 0.271 & 0.259 & 0.080 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 98 & 0.269 & 0.178 & 0.202 & 88 & 1.233 & 0.169 & 1.222 & NA & 

In [26]:
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    for iv_strength in [.2, .5]:
        Z, X, Y, true_fn = get_data(
            1000000, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5)
        true = np.mean(moment_fn(X, true_fn))
        for n in [500, 1000, 2000]:
            print(n, fname, iv_strength, true)
            results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, None, None,
                                                                  fname=fname,
                                                                  iv_strength=iv_strength)
                                                            for it in range(100))
            joblib.dump((true, results), f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            display(df)

500 abs 0.2 -0.0002851219415974534


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


,dr,tmle,ipw,direct
cov,83,88,NA,NA
rmse,0.175,10.625,0.104,0.158
bias,0.134,1.157,0.072,0.137
std,0.112,10.562,0.075,0.079


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


1000 abs 0.2 -0.0002851219415974534


...

,dr,tmle,ipw,direct
cov,97,99,NA,NA
rmse,0.049,0.316,0.046,0.025
bias,0.001,0.012,0.010,0.006
std,0.049,0.316,0.045,0.025


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2000 sin 0.5 0.14968064900373804


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished


,dr,tmle,ipw,direct
cov,98,100,NA,NA
rmse,0.035,0.260,0.026,0.024
bias,0.007,0.013,0.000,0.007
std,0.034,0.259,0.026,0.023


In [30]:
def get_result_dict(results, true, alpha=0.99):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        confidence = alpha
        data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov': cov,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}'}
    return df

In [31]:
res = {}
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    res[fname] = {}
    for n in [500, 1000, 2000]:
        res[fname][f'$n={n}$'] = {}
        for iv_strength in [.2, .5]:
            true, results = joblib.load(f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            res[fname][f'$n={n}$'][f'$\\rho={iv_strength}$'] = df
        res[fname][f'$n={n}$'] = pd.concat(res[fname][f'$n={n}$'], sort=False)
    res[fname] = pd.concat(res[fname], sort=False)
res = pd.concat(res, sort=False).unstack(level=3)

In [32]:
res

dr               tmle                ipw         \
                             cov   rmse   bias  cov    rmse   bias cov   rmse   
abs     $n=500$  $\rho=0.2$   83  0.175  0.134   88  10.625  1.157  NA  0.104   
                 $\rho=0.5$   99  0.087  0.016   99   1.735  0.134  NA  0.073   
        $n=1000$ $\rho=0.2$   93  0.115  0.076   96   0.199  0.068  NA  0.086   
                 $\rho=0.5$   99  0.061  0.004   99   0.340  0.009  NA  0.053   
        $n=2000$ $\rho=0.2$   96  0.077  0.029   98   0.193  0.026  NA  0.058   
                 $\rho=0.5$   97  0.044  0.010  100   0.281  0.022  NA  0.036   
2dpoly  $n=500$  $\rho=0.2$   93  0.169  0.124   96  14.081  1.491  NA  0.250   
                 $\rho=0.5$   97  0.112  0.007  100   2.412  0.266  NA  0.117   
        $n=1000$ $\rho=0.2$   91  0.130  0.072   94   0.243  0.030  NA  0.184   
                 $\rho=0.5$   99  0.074  0.001   99   0.309  0.012  NA  0.081   
        $n=2000$ $\rho=0.2$   95  0.089  0.033   98   0.223  0.041  NA  0.120   
                 $\rho=0.5$   97  0.063  0.020   98   0.242  0.029  NA  0.067   
sigmoid $n=500$  $\rho=0.2$   84  0.141  0.105   90   5.208  0.556  NA  0.068   
                 $\rho=0.5$  100  0.070  0.017   99   1.074  0.055  NA  0.066   
        $n=1000$ $\rho=0.2$   94  0.087  0.053   98   0.176  0.051  NA  0.064   
                 $\rho=0.5$   99  0.050  0.007   99   0.330  0.025  NA  0.050   
        $n=2000$ $\rho=0.2$   96  0.059  0.020   99   0.130  0.011  NA  0.047   
                 $\rho=0.5$   99  0.034  0.005   99   0.262  0.016  NA  0.029   
sin     $n=500$  $\rho=0.2$   80  0.157  0.124   87   4.982  0.559  NA  0.076   
                 $\rho=0.5$  100  0.063  0.007  100   1.021  0.041  NA  0.058   
        $n=1000$ $\rho=0.2$   93  0.098  0.067   94   0.180  0.060  NA  0.068   
                 $\rho=0.5$   97  0.049  0.001   99   0.316  0.012  NA  0.046   
        $n=2000$ $\rho=0.2$   95  0.063  0.026   97   0.135  0.016  NA  0.047   
                 $\rho=0.5$   98  0.035  0.007  100   0.260  0.013  NA  0.026   

                                   direct                
                              bias    cov   rmse   bias  
abs     $n=500$  $\rho=0.2$  0.072     NA  0.158  0.137  
                 $\rho=0.5$  0.016     NA  0.053  0.004  
        $n=1000$ $\rho=0.2$  0.053     NA  0.101  0.085  
                 $\rho=0.5$  0.007     NA  0.036  0.009  
        $n=2000$ $\rho=0.2$  0.019     NA  0.070  0.051  
                 $\rho=0.5$  0.005     NA  0.034  0.008  
2dpoly  $n=500$  $\rho=0.2$  0.232     NA  0.186  0.159  
                 $\rho=0.5$  0.074     NA  0.093  0.040  
        $n=1000$ $\rho=0.2$  0.166     NA  0.133  0.108  
                 $\rho=0.5$  0.043     NA  0.064  0.024  
        $n=2000$ $\rho=0.2$  0.101     NA  0.091  0.068  
                 $\rho=0.5$  0.041     NA  0.052  0.023  
sigmoid $n=500$  $\rho=0.2$  0.006     NA  0.118  0.100  
                 $\rho=0.5$  0.039     NA  0.044  0.014  
        $n=1000$ $\rho=0.2$  0.002     NA  0.070  0.050  
                 $\rho=0.5$  0.021     NA  0.028  0.008  
        $n=2000$ $\rho=0.2$  0.015     NA  0.048  0.029  
                 $\rho=0.5$  0.007     NA  0.023  0.001  
sin     $n=500$  $\rho=0.2$  0.033     NA  0.135  0.120  
                 $\rho=0.5$  0.023     NA  0.037  0.002  
        $n=1000$ $\rho=0.2$  0.025     NA  0.082  0.065  
                 $\rho=0.5$  0.010     NA  0.025  0.006  
        $n=2000$ $\rho=0.2$  0.002     NA  0.056  0.039  
                 $\rho=0.5$  0.000     NA  0.024  0.007

In [33]:
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='lll||lll|lll|lll|lll|',
                   multicolumn_format='c|'))

\begin{tabular}{lll||lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{6}{*}{\textbf{abs}} & \multirow[t]{2}{*}{\textbf{$n=500$}} & \textbf{$\rho=0.2$} & 83 & 0.175 & 0.134 & 88 & 10.625 & 1.157 & NA & 0.104 & 0.072 & NA & 0.158 & 0.137 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 99 & 0.087 & 0.016 & 99 & 1.735 & 0.134 & NA & 0.073 & 0.016 & NA & 0.053 & 0.004 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=1000$}} & \textbf{$\rho=0.2$} & 93 & 0.115 & 0.076 & 96 & 0.199 & 0.068 & NA & 0.086 & 0.053 & NA & 0.101 & 0.085 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 99 & 0.061 & 0.004 & 99 & 0.340 & 0.009 & NA & 0.053 & 0.007 & NA & 0.036 & 0.009 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=2000$}} & \textbf{$\rho=0.2$} & 96 & 0.077 & 0.029 & 98 & 0.193 &